In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-df/train_age_dataset.csv


In [2]:
df=pd.read_csv('/kaggle/input/train-df/train_age_dataset.csv')
df.head(2)

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,...,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
0,265153,48958844,2,1,0.0,0.0,0.0,0,0.0,0.0,...,0.20000,0.0,0.041667,0.025000,0.0,0.000000,0.175,0.033333,0.0,1
1,405231,51100441,2,2,0.0,0.0,0.0,0,0.0,0.0,...,0.09322,0.0,0.012712,0.018644,0.0,0.084746,0.000,0.033898,82.5,2


In [3]:
df.drop(['Unnamed: 0','userId'],axis=1,inplace=True)

In [4]:
df.shape[1]

25

In [5]:
from xgboost import XGBClassifier

In [6]:
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [7]:
X=df.drop(['age_group'],axis=1)
y=df['age_group']

In [8]:
train_x,test_x,train_y,test_y=train_test_split(X,y,test_size=.0625,random_state=42)

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler_fit = scaler.fit(X)

X_train = scaler_fit.transform(train_x)
X_test = scaler_fit.transform(test_x)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(train_y)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_y)
class_weights = dict(zip(classes, weights))
class_weights

In [10]:
xgb_clf = XGBClassifier(tree_method = 'gpu_hist', gpu_id=0, objective = "multi:softmax",learning_rate=0.01, max_depth=15, subsample=0.8, n_estimators=750, reg_lambda=5, num_class = 4, class_weights=class_weights)

In [12]:
xgb_clf.fit(X_train,train_y)

XGBClassifier(gpu_id=0, learning_rate=0.01, max_depth=20, n_estimators=1000,
              num_class=4, objective='multi:softprob', tree_method='gpu_hist')

In [13]:
xgb_clf.score(X_train,train_y)

0.9984927858270192

In [14]:
xgb_clf.score(X_test,test_y)

0.7462362952053674

In [15]:
expected_y  = test_y
predicted_y = xgb_clf.predict(X_test)
print('CatBoost: ')
print(metrics.classification_report(expected_y, predicted_y))

CatBoost: 
              precision    recall  f1-score   support

           1       0.90      0.90      0.90     15554
           2       0.49      0.56      0.52      2968
           3       0.45      0.48      0.46      2976
           4       0.48      0.37      0.42      2946

    accuracy                           0.75     24444
   macro avg       0.58      0.58      0.58     24444
weighted avg       0.75      0.75      0.75     24444



In [16]:
metrics.confusion_matrix(expected_y, predicted_y)

array([[14035,   657,   507,   355],
       [  554,  1674,   445,   295],
       [  454,   566,  1437,   519],
       [  489,   530,   832,  1095]])